In [ ]:
set.seed(12345)
wood <- rnorm(24)
heat <- rnorm(24)
mean(wood)
mean(heat)
sd(wood)
sd(heat)

In [ ]:
plot(wood,heat, main=NULL) 

In [ ]:
plot(wood,(wood-mean(wood)), main=NULL)

In [ ]:
cpWH <- wood * heat
hist(cpWH, main=NULL)


In [ ]:
mean(cpWH)

In [ ]:
# Make a new, fake version of heat that will correlate with wood
newHeat <- wood/1.41 + heat/1.41   # Make a mixture of the two old variables
mean(newHeat)                      # What's the mean of our new heat variable?
sd(newHeat)                        # What's the sd of our new heat variable?

In [ ]:
cpWnewH <- wood * newHeat
hist(cpWnewH, main=NULL)

In [ ]:
mean(cpWnewH)

In [ ]:
plot(wood,newHeat, main=NULL) 

In [ ]:
cor(wood,newHeat)

In [ ]:
set.seed(12345)                  # Start with a random number seed
wood <- rnorm(2400)              # Make two vectors of N=2400
heat <- rnorm(2400)
fireDF <- data.frame(wood, heat) # Put them in a dataframe
nrow(fireDF)                     # Verifying 2400 rows of two variables
fireDF[sample(nrow(fireDF), 24), ] # Generates one sample of n=24  

In [ ]:
cor(fireDF[sample(nrow(fireDF), 24), ])
cor(fireDF[sample(nrow(fireDF), 24), ])[1,2]

In [ ]:
corDist <- replicate(5000,cor(fireDF[sample(nrow(fireDF), 24), ])[1,2])
hist(corDist, main=NULL)


In [ ]:
mean(corDist)

In [ ]:
newHeat <- wood/1.41 + heat/1.41
newfireDF <- data.frame(wood, newHeat) # Put them in a dataframe
newcorDist <- replicate(5000,cor(newfireDF[sample(nrow(newfireDF), 24), ])[1,2],simplify=TRUE)
hist(newcorDist, main=NULL)


In [ ]:
mean(newcorDist)

In [ ]:
# Conduct a null hypothesis test on one correlation
set.seed(12345)
wood <- rnorm(24)
heat <- rnorm(24)
cor.test(wood,heat)

cor.test(wood,(wood/1.41 + heat/1.41))

In [ ]:
cor.test(iris[,"Sepal.Width"],iris[,"Petal.Width"])

In [ ]:
# install.packages("BayesFactor")
library("BayesFactor")


In [ ]:
bfCorTest <- function (x,y) # Get r from BayesFactor
{
  zx <- scale(x) # Standardize X
  zy <- scale(y) # Standardize Y
  zData <- data.frame(x=zx,rhoNot0=zy) # Put in a data frame
  bfOut <- generalTestBF(x ~ rhoNot0, data=zData) # linear coefficient
  mcmcOut <- posterior(bfOut,iterations=10000) # posterior samples
  print(summary(mcmcOut[,"rhoNot0"])) # Get the HDI for r
  return(bfOut) # Return Bayes factor object
}

In [ ]:
set.seed(12345)
wood <- rnorm(24)
heat <- rnorm(24)
bfCorTest(wood,heat)

In [ ]:
newHeat <- wood/1.41 + heat/1.41
bfCorTest(newHeat, wood)

In [ ]:
bfCorTest(iris[,"Sepal.Width"],iris[,"Petal.Width"])

In [ ]:
# Chi-square section

make2x2table <- function(ul) # The user supplies the count for the upper left cell
{
  ll <- 50 - ul # Calculate the lower left cell
  ur <- 30 - ul # Calculate the upper right cell
  lr <- 50 - ur # Calculate the lower right cell
  
  # Put all of the cells into a 2x2 matrix
  matrix(c(ul,ur,ll,lr), nrow=2, ncol=2, byrow=TRUE)
}


In [ ]:
make2x2table(15)  # Should be like Table 7.2
make2x2table(0)   # Should be like Table 7.3
make2x2table(30)  # Should be like Table 7.4

In [ ]:
calcChiSquared <- function(actual, expected) # Calculate chi-squared
{
  diffs <- actual - expected        # Take the raw difference for each cell
  diffsSq <- diffs ^ 2              # Square each cell
  diffsSqNorm <- diffsSq / expected # Normalize with expected cells
  
  sum(diffsSqNorm)                  # Return the sum of the cells
}

In [ ]:
# This makes a matrix that is just like Table 7.2
# This table represents the null hypothesis of independence
expectedValues <- matrix(c(15,15,35,35), nrow=2, ncol=2, byrow=TRUE)

calcChiSquared(make2x2table(15),expectedValues)
calcChiSquared(make2x2table(0),expectedValues)
calcChiSquared(make2x2table(30),expectedValues)

In [ ]:
set.seed(12)

hist(rbinom(1000,30,prob=0.5), main=NULL)

In [ ]:
mean(rbinom(1000,30,prob=0.5))

In [ ]:
chiDist <- replicate(100000,calcChiSquared(make2x2table(rbinom(n=1,size=30,prob=0.5)),expectedValues))
hist(chiDist, main=NULL)



In [ ]:
quantile(chiDist,probs=c(0.95))

In [ ]:
calcChiSquared(make2x2table(20),expectedValues)

calcChiSquared(make2x2table(10),expectedValues)

In [ ]:
# Run the chi-square test on Table 7.1 data
chisq.test(make2x2table(20), correct=FALSE)

In [ ]:
# Run the chi-square test on Table 7.1 data
# data(Titanic) # Should not need this
badBoatMF <- ftable(Titanic, row.vars=2, col.vars="Survived")
badBoatMF
chisq.test(badBoatMF, correct=FALSE)

In [ ]:
# Bayesian approach using BayesFactor package

ctBFout <- contingencyTableBF(make2x2table(20),sampleType="poisson",posterior=FALSE)
ctBFout

ctMCMCout <- contingencyTableBF(make2x2table(20),sampleType="poisson",posterior=TRUE,iterations=10000)
summary(ctMCMCout)
downProp <- ctMCMCout[,"lambda[1,1]"]/ctMCMCout[,"lambda[2,1]"]



In [ ]:
hist(downProp, main=NULL)

In [ ]:
upProp <- ctMCMCout[,"lambda[1,2]"]/ctMCMCout[,"lambda[2,2]"]


In [ ]:
hist(upProp, main=NULL)

In [ ]:
diffProp <- downProp-upProp

hist(diffProp, main=NULL)
abline(v=quantile(diffProp,c(0.025)), col="black")
abline(v=quantile(diffProp,c(0.975)), col="black")




In [ ]:
mean(diffProp)

In [ ]:
badBoatMF <- ftable(Titanic, row.vars=2, col.vars="Survived")
ctBFout <- contingencyTableBF(badBoatMF,sampleType="poisson",posterior=FALSE)
ctBFout

ctMCMCout <- contingencyTableBF(badBoatMF,sampleType="poisson",posterior=TRUE,iterations=10000)
summary(ctMCMCout)
maleProp <- ctMCMCout[,"lambda[1,1]"]/ctMCMCout[,"lambda[1,2]"]
femaleProp <- ctMCMCout[,"lambda[2,1]"]/ctMCMCout[,"lambda[2,2]"]
diffProp <- maleProp - femaleProp


In [ ]:
hist(diffProp)

abline(v=quantile(diffProp,c(0.025)), col="black")
abline(v=quantile(diffProp,c(0.975)), col="black")

In [ ]:
mean(diffProp)